# Modelling PI-ANNA questions

In [13]:
import scipy.io
import os
import numpy as np

import sumProduct as sp
import graphviz as gp
import matplotlib.pyplot as plt
import subprocess

import autograd as ag
import autograd.numpy as anp
import autograd.scipy.special as spf
import scipy.optimize as opt

Hiermit suchen wir jetzt das P(Pain) das dazu führt dass der Patient im chronic pain state landet - welche Werte müssen für P(Pain) und P(Nopain)

Die ersten beiden Werte: pain und nopain, der zweite: residuals

In [14]:
A = np.array([[0.5, 1-.5],[0.2, 1-0.2],[0.35, 1-0.35],
             [0.07, 1-0.07],[0.05, 1-0.05],[0.04, 1-0.04],
             [0.1, 1-0.1],[0.1, 1-0.1],[0.1, 1-0.1],
             [0.15, 1-0.15],[0.5, 0.5],[0.03, 1-0.03],
             [0.03, 1-0.03],[0.03, 1-0.03],[0.04, 1-0.04],
             [0.1, 1-0.1],[0.05, 1-0.05],[0.1, 1-0.1],
             [0.0,0.0],[0.0,0.0],[0.0,0.0],
             [0.0,0.0],[0.03,1-0.03],[0.0,0.0],
             [0.18,1-0.18],[0.33,1-0.33],[0.31,1-0.31],
             [0.22,1-0.22],[0.35,1-0.35],[0.0,0.0],
             [0.69,1-0.69],[0.73,1-0.73],[0.53,1-0.53],
             [0.25,1-0.25],[0.3,1-0.3],[0.3,1-0.3],
             [0.0,0.0],[0.0,0.0],[0.0,0.0],
             [0.2,1-0.2],[0.45,1-0.45],[0.7,1-0.7],
             [0.07,1-0.07],[0.13,1-0.13],[0.13,1-0.13],
             [0.03,1-0.03],[0.03,1-0.03],[0.16,1-0.16],
             [0.0,0.0],[0.5,0.5],[0.5,0.5],
             [0.07,1-0.07],[0.04,1-0.04],[0.01,1-0.01],
             [0.36,1-0.36],[0.2,1-0.2],[0.39,1-0.39],
             [0.01, 1-0.01],[.32,1-0.32],[0.31,1-0.31],
             [0.53, 1-0.53],[0.38,1-0.38],[0.3,1-0.3],
             [0.92,1-0.92],[0.48,1-0.48],[0.45,1-0.45],
             [0.88,1-0.88],[0.83,1-0.83],[0.93,1-0.93],
             [0.04,1-0.04],[.82,1-0.82],[.74,1-.74],
             [0.53,1-0.53],[0.73,1-0.73],[0.77,1-0.77],
             [0.25,1-0.25],[0.56,1-0.56],[0.24,1-0.24],
             [0,0],[0,0],[.43,1-.43],
             [.88,1-.88],[.91,1-.91],[.93,1-.93],
             [.31,1-.31],[.75,1-.75],[.48,1-.48],
             [.03,1-0.03],[.31,1-.31],[.17,1-.17],
             [.04,1-.04],[.13,1-.13],[.19,1-.19],
             [.18,1-.18],[.44,1-.44],[.29,1-.29],
             [1.0,0.0],[1.0,0.0],[1.0,0.0],
             [.12,1-.12],[.43,1-.43],[0.0,0.0],
             [.99,1-.99],[.54,1-.54],[.36,1-.36]])

In [15]:
A

array([[0.5 , 0.5 ],
       [0.2 , 0.8 ],
       [0.35, 0.65],
       [0.07, 0.93],
       [0.05, 0.95],
       [0.04, 0.96],
       [0.1 , 0.9 ],
       [0.1 , 0.9 ],
       [0.1 , 0.9 ],
       [0.15, 0.85],
       [0.5 , 0.5 ],
       [0.03, 0.97],
       [0.03, 0.97],
       [0.03, 0.97],
       [0.04, 0.96],
       [0.1 , 0.9 ],
       [0.05, 0.95],
       [0.1 , 0.9 ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.03, 0.97],
       [0.  , 0.  ],
       [0.18, 0.82],
       [0.33, 0.67],
       [0.31, 0.69],
       [0.22, 0.78],
       [0.35, 0.65],
       [0.  , 0.  ],
       [0.69, 0.31],
       [0.73, 0.27],
       [0.53, 0.47],
       [0.25, 0.75],
       [0.3 , 0.7 ],
       [0.3 , 0.7 ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.  , 0.  ],
       [0.2 , 0.8 ],
       [0.45, 0.55],
       [0.7 , 0.3 ],
       [0.07, 0.93],
       [0.13, 0.87],
       [0.13, 0.87],
       [0.03, 0.97],
       [0.03, 0.97],
       [0.16,

In [16]:
a= np.array([[.5,1-.5],[1,0],[0,1],[.5,1-.5],[1,0],[0,1],[.5,1-.5],[1,0],[0,1]])
b = np.array([[1],[0],[.1],[1],[0],[.1],[1],[0],[.1]])
print(a)
print(b)
P=np.linalg.lstsq(a,b)[0]
print(P)

[[0.5 0.5]
 [1.  0. ]
 [0.  1. ]
 [0.5 0.5]
 [1.  0. ]
 [0.  1. ]
 [0.5 0.5]
 [1.  0. ]
 [0.  1. ]]
[[1. ]
 [0. ]
 [0.1]
 [1. ]
 [0. ]
 [0.1]
 [1. ]
 [0. ]
 [0.1]]
[[0.31666667]
 [0.41666667]]


C:\Users\OEM\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


In [17]:
B = np.array([[0.2],[0.35],[.7],
              [0.05],[0.04],[0.03],
              [0.1],[0.1],[0.1],
              [.5],[.03],[.0],
              [.03],[.04],[.03],
              [.05],[.1],[.1],
              [0.0],[0.0],[0.0],
              [0.03],[0.0],[0.2],
              [.33],[.31],[.44],
              [.35],[.0],[.0],
              [.73],[.53],[.38],
              [.3],[.3],[.33],
              [0],[0],[0],
              [.45],[.7],[.5],
              [.13],[.13],[.14],
              [.03],[.16],[.06],
              [.5],[.5],[.2],
              [.04],[.01],[.13],
              [.2],[.39],[.42],
              [.32],[.31],[.72],
              [.38],[.3],[.69],
              [.48],[.45],[.80],
              [.83],[.93],[.46],
              [.82],[.74],[.96],
              [.73],[.77],[.77],
              [.56],[.24],[.08],
              [.0],[.43],[.93],
              [.91],[.93],[.99],
              [.75],[.48],[.92],
              [.31],[.17],[.02],
              [.13],[.19],[.01],
              [.44],[.29],[.92],
              [1],[1],[1],
              [.43],[.0],[.12],
              [.54],[.36],[.98]])

In [18]:
B

array([[0.2 ],
       [0.35],
       [0.7 ],
       [0.05],
       [0.04],
       [0.03],
       [0.1 ],
       [0.1 ],
       [0.1 ],
       [0.5 ],
       [0.03],
       [0.  ],
       [0.03],
       [0.04],
       [0.03],
       [0.05],
       [0.1 ],
       [0.1 ],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.03],
       [0.  ],
       [0.2 ],
       [0.33],
       [0.31],
       [0.44],
       [0.35],
       [0.  ],
       [0.  ],
       [0.73],
       [0.53],
       [0.38],
       [0.3 ],
       [0.3 ],
       [0.33],
       [0.  ],
       [0.  ],
       [0.  ],
       [0.45],
       [0.7 ],
       [0.5 ],
       [0.13],
       [0.13],
       [0.14],
       [0.03],
       [0.16],
       [0.06],
       [0.5 ],
       [0.5 ],
       [0.2 ],
       [0.04],
       [0.01],
       [0.13],
       [0.2 ],
       [0.39],
       [0.42],
       [0.32],
       [0.31],
       [0.72],
       [0.38],
       [0.3 ],
       [0.69],
       [0.48],
       [0.45],
       [0.8 ],
       [0.

In [19]:
len(B)

105

In [20]:
105/35

3.0

In [21]:
P = np.linalg.lstsq(A,B)[0]

C:\Users\OEM\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if __name__ == '__main__':


In [22]:
P

array([[0.87493722],
       [0.12563615]])

In [23]:
C = np.dot(A,P)
len(C)

105


### 1) Matrix bauen aus allen Versuchspersonen. $\checkmark$

### 2) Vektor bauen $\checkmark$

### 3) least square $\checkmark$    Besser: beta regression!

### 4) hidden factors diese Probs einfügen

### 5) laufen lassen bzw. therapieren

# Model

In [12]:
vlist=['H'+str(i) for i in range(4)]
hidden=[sp.variableNode(name, vrange=['pain', 'nopain']) for name in vlist]
Slist=['S'+str(i) for i in range(4)]
sensory=[sp.variableNode(name, vrange=['tickle','noci']) for name in Slist]
variables=sensory+hidden
print(len(variables))
factors=[]

8


In [13]:
prior=sp.freeEnergyFactorNode()
prior.addNeighbour(hidden[0])

def exprior(condrv,conditioners):
    if condrv[0] is None:
        return 500.0
    if condrv[0]=='nopain':
        return 0.2
    if condrv[0]=='pain':
        return 0.8
    
prior.setPrior(exprior)
factors.append(prior)
prior.predictFactor()



# Hier kommen die PkeepPain und PkeepNoPain aus den Daten rein
def hiddenprior(condrv,conditioners):
    print(condrv,conditioners),
    if condrv[0] is None:
        return 200.0
    if conditioners[0]=='pain':
        if condrv[0]=='pain':
            return 0.9
        if condrv[0]=='nopain':
            return 0.1
    else:
        if condrv[0]=='pain':
            return 0.9
        if condrv[0]=='nopain':
            return 0.1

        
#Hier kommen die P(sensation|pain) rein aus den Daten
def sensprior(condrv,conditioners):
    print(condrv,conditioners)
    if condrv[0] is None:
        return 1000.0
    if conditioners[0]=='pain':
        if condrv[0]=='noci':
            return 0.9
        if condrv[0]=='tickle':
            return 0.1
    else:
        if condrv[0]=='tickle':
            return 0.9
        if condrv[0]=='noci':
            return 0.1        

In [14]:
parent=hidden[0]
hiddenfactors=[]
for child in hidden[1:]:
    fe=sp.freeEnergyFactorNode()
    fe.addNeighbour(child,isConditioner=False)
    fe.addNeighbour(parent,isConditioner=True)
    fe.setPrior(hiddenprior)
    factors.append(fe)
    hiddenfactors.append(fe)
    parent=child
    fe.predictFactor()
print(len(factors))
print(len(hiddenfactors))

(None,) ('pain',)
('pain',) ('pain',)
('nopain',) ('pain',)
(None,) ('nopain',)
('pain',) ('nopain',)
('nopain',) ('nopain',)
(None,) ('pain',)
('pain',) ('pain',)
('nopain',) ('pain',)
(None,) ('nopain',)
('pain',) ('nopain',)
('nopain',) ('nopain',)
(None,) ('pain',)
('pain',) ('pain',)
('nopain',) ('pain',)
(None,) ('nopain',)
('pain',) ('nopain',)
('nopain',) ('nopain',)
4
3


In [15]:
sensfactors=[]
for h,s in zip(hidden,sensory):
    f=sp.freeEnergyFactorNode()
    f.addNeighbour(s)
    f.addNeighbour(h,isConditioner=True)
    f.setPrior(sensprior)
    factors+=[f]
    sensfactors+=[f]
    f.predictFactor()
    
allNodes=variables+factors

(None,) ('pain',)
('tickle',) ('pain',)
('noci',) ('pain',)
(None,) ('nopain',)
('tickle',) ('nopain',)
('noci',) ('nopain',)
(None,) ('pain',)
('tickle',) ('pain',)
('noci',) ('pain',)
(None,) ('nopain',)
('tickle',) ('nopain',)
('noci',) ('nopain',)
(None,) ('pain',)
('tickle',) ('pain',)
('noci',) ('pain',)
(None,) ('nopain',)
('tickle',) ('nopain',)
('noci',) ('nopain',)
(None,) ('pain',)
('tickle',) ('pain',)
('noci',) ('pain',)
(None,) ('nopain',)
('tickle',) ('nopain',)
('noci',) ('nopain',)


In [16]:
outf=open("PiANNA.dot","w")
outf.write(hidden[0].dotPrint())
outf.close()
subprocess.call("neato -T png -o PiANNA.png PiANNA.dot", shell=True)

0

# Beta regression

In [2]:
def betaprob(pred_values,lam,nu):
    """log-likelihood of lambda and nu for beta-distributed pred_values. 
    lam=alpha/(alpha+beta), nu=alpha+beta"""
    return np.sum(-spf.betaln(lam*nu,(1.0-lam)*nu)+(lam*nu-1.0)*np.log(pred_values+1e-10)+((1.0-lam)*nu-1.0)*np.log(1.0-pred_values+1e-10))

In [3]:
def time_series_prediction(values,persistence_probs):
    """Likelihood of time series. values[instance,time] \in [0,1] are the pain probability values to be predicted
    persistence_probs: probabilities of pain / nopain persistence
    nu: pseudocount of beta-distribution for likelihood"""
    
    predictions=[]
    for instance in values:
        predicted_values=[]
        for time_point in range(1,len(instance)):
            ppain=instance[time_point-1]
            # Markov chain prediction
            predicted_values.append(ppain*persistence_probs[0]+(1.0-ppain)*(1.0-persistence_probs[1]))
        predictions.append(predicted_values)
        
    return np.array(predictions)

In [4]:
def time_series_likelihood(values,persistence_probs,nu):
    """Likelihood of time series. values[instance,time] \in [0,1] are the pain probability values to be predicted
    persistence_probs: probabilities of pain / nopain persistence
    nu: pseudocount of beta-distribution for likelihood"""
    
    predictions=time_series_prediction(values,persistence_probs)

    return np.sum(betaprob(values[:,1:].flatten(),predictions.flatten(),nu))

In [5]:
def cost_function_and_gradient(values,nu):
    
    costfunc=lambda persistence_probs:-time_series_likelihood(values,persistence_probs,nu)
    
    costfunc_grad=ag.grad(costfunc)
    
    return lambda persistence_probs:(costfunc(persistence_probs),costfunc_grad(persistence_probs))

In [24]:
# Tanja's participants
participants=A/100.0

In [25]:
# starting values for persistence probabilities of pain and nopain
pprobs=anp.array([0.5,0.99])

In [26]:
for values in participants:

    cgf=cost_function_and_gradient(values[np.newaxis,...],1000.0)
    best_p,best_ll,d=opt.fmin_l_bfgs_b(cgf,pprobs,bounds=[(0.01,0.99)]*len(pprobs),factr=1000.0,pgtol=1e-8)
    
    predictions=time_series_prediction(values[anp.newaxis,...],best_p)
    
    print("Best transition probabilities",best_p,"Actual and predicted pain values",values[1:],predictions)

TypeError: ufunc 'betaln' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''